### Setup the environment to run juypter and django
Run this block of code first

In [13]:
import os,sys
import django
from django.db import models
from django.db.models import Subquery, OuterRef
from django.db.models import Q

PWD = os.path.dirname(os.path.dirname(os.path.abspath("test-1.ipynb")))
print(f"PWD={PWD}\\")
sys.path.insert(0, f"{PWD}")

# wsqi.py
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "beer.settings")
#  Allow queryset filtering asynchronously when running in a Jupyter notebook
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()
from django.contrib.auth.models import User
from nascar.models import Bet, Race, Track, Driver

PWD=c:\Users\me\Documents\VisualCodeSource\beerme1219\beer\


### Get the race schedule for 2024

In [14]:

# Race.objects.get(pk=1)
# https://docs.djangoproject.com/en/5.0/ref/models/expressions/
# print(Track.objects.get(pk=track_pk))

race_schedule = Race.objects.filter(race_date__gt="2024-03-01")
# print the actual sql code
print(race_schedule.query)
# race_schedule = Race.objects.filter(bet=OuterRef("pk"))
for rs in race_schedule:
    print(rs)

SELECT `race`.`id`, `race`.`date_created`, `race`.`user_id`, `race`.`date_updated`, `race`.`race_name`, `race`.`race_date`, `race`.`Race Time`, `race`.`track_id`, `race`.`TV/RADIO` FROM `race` WHERE `race`.`race_date` > 2024-03-01 ORDER BY `race`.`race_date` ASC
Las Vegas Motor Speedway Pennzoil 400 - 2024-03-03
Phoenix Raceway Shriners Children's 500 - 2024-03-10
Bristol Motor Speedway NASCAR Cup Series at Bristol - 2024-03-17
Circuit Of The Americas NASCAR Cup Series at Circuit of the Americas - 2024-03-24
Richmond Raceway NASCAR Cup Series at Richmond - 2024-03-31
Martinsville Speedway NASCAR Cup Series at Martinsville - 2024-04-07
Texas Motor Speedway NASCAR Cup Series at Texas - 2024-04-14


### Get all the bets

In [15]:
# https://books.agiliq.com/projects/django-orm-cookbook/en/latest/subquery.html
# make sure to the the code block above first
sq = Race.objects.filter(track=Subquery(race_schedule.values("track"))).values(
    "track"
)
# print(sq)
# show the sql query
print(sq.query)
# get all bets
bets = Bet.objects.select_related("race")
# print all bets
for b in bets:
    print(f"{b.finish} {b.player} {b.race} {b.driver}")

SELECT `race`.`track_id` FROM `race` WHERE `race`.`track_id` = (SELECT U0.`track_id` FROM `race` U0 WHERE U0.`race_date` > 2024-03-01 ORDER BY U0.`race_date` ASC) ORDER BY `race`.`race_date` ASC
32 Greg Daytona International Speedway Daytona 500 - 2024-02-18 22 - Joey Logano / Team Penske
14 Bob Daytona International Speedway Daytona 500 - 2024-02-18 9 - Chase Elliott / Hendrick Motorsports
28 Bob Atlanta Motor Speedway Ambetter Health 400 - 2024-02-25 22 - Joey Logano / Team Penske
32 Greg Atlanta Motor Speedway Ambetter Health 400 - 2024-02-25 5 - Kyle Larson / Hendrick Motorsports
5 Greg Las Vegas Motor Speedway Pennzoil 400 - 2024-03-03 12 - Ryan Blaney / Team Penske
18 Bob Las Vegas Motor Speedway Pennzoil 400 - 2024-03-03 24 - William Byron / Hendrick Motorsports


### Get All Drivers who names start with 'R' or 'J'

In [16]:

print(Driver.objects.filter(name__startswith='R') | Driver.objects.filter(name__startswith='J'))

Driver.objects.filter(Q(name__startswith="R") | Q(name__startswith="N")).order_by('name')

<QuerySet [<Driver: 1 - Ross Chastain / Trackhouse Racing>, <Driver: 12 - Ryan Blaney / Team Penske>, <Driver: 51 - J.J. Yeley / In-Active>, <Driver: 15 - Riley Herbst / In-Active>, <Driver: 22 - Joey Logano / Team Penske>, <Driver: 31 - Justin Haley / Rick Ware Racing>, <Driver: 41 - Ryan Preece / Stewart-Haas Racing>, <Driver: 47 - Ricky Stenhouse Jr. / JTG Daugherty Racing>, <Driver: 84 - Jimmie Johnson / Legacy Motor Club>, <Driver: 9 - Josh Berry / Stewart-Haas Racing>, <Driver: 42 - John Hunter Nemechek / Legacy Motor Club>]>


<QuerySet [<Driver: None - N/A / In-Active>, <Driver: 42 - Noah Gragson / Stewart-Haas Racing>, <Driver: 47 - Ricky Stenhouse Jr. / JTG Daugherty Racing>, <Driver: 15 - Riley Herbst / In-Active>, <Driver: 1 - Ross Chastain / Trackhouse Racing>, <Driver: 12 - Ryan Blaney / Team Penske>, <Driver: 41 - Ryan Preece / Stewart-Haas Racing>]>

### Get race dates greater than 1/1/2024 and track names that begin with a 'D' | 'R' | 'P'
https://books.agiliq.com/projects/django-orm-cookbook/en/latest/and_query.html

In [17]:
qs1 = Race.objects.filter(track__track_name__startswith='D', race_date__gt = '2024-01-01')
qs2 = Race.objects.filter(track__track_name__startswith='D') & Race.objects.filter( race_date__gt = '2024-01-01')
qs3 = Race.objects.filter(Q(track__track_name__startswith='D') & Q( race_date__gt = '2024-01-01'))
print(qs1)
print(qs2)
print(qs3)


<QuerySet [<Race: Daytona International Speedway Daytona 500 - 2024-02-18>]>
<QuerySet [<Race: Daytona International Speedway Daytona 500 - 2024-02-18>]>
<QuerySet [<Race: Daytona International Speedway Daytona 500 - 2024-02-18>]>


### How to do a NOT query in Django queryset?
https://books.agiliq.com/projects/django-orm-cookbook/en/latest/notequal_query.html

In [18]:
# AJ Allmendinger is gone
qs = Driver.objects.filter(~Q(name__startswith='A')).order_by("name")
qs

<QuerySet [<Driver: 78 - B.J. McLeod / Live Fast Motorsports>, <Driver: 6 - Brad Keselowski / RFK Racing>, <Driver: 23 - Bubba Wallace / 23XI Racing>, <Driver: 77 - Carson Hocevar / Spire Motorsports>, <Driver: 14 - Chase Briscoe / Stewart-Haas Racing>, <Driver: 9 - Chase Elliott / Hendrick Motorsports>, <Driver: 17 - Chris Buescher / RFK Racing>, <Driver: 20 - Christopher Bell / Joe Gibbs Racing>, <Driver: 51 - Cody Ware / Rick Ware Racing>, <Driver: 50 - Conor Daly / In-Active>, <Driver: 7 - Corey LaJoie / Spire Motorsports>, <Driver: 31 - Daniel Hemric / Kaulig Racing>, <Driver: 99 - Daniel Suarez / Trackhouse Racing>, <Driver: 11 - Denny Hamlin / Joe Gibbs Racing>, <Driver: 43 - Erik Jones / Legacy Motor Club>, <Driver: 21 - Harrison Burton / Wood Brothers Racing>, <Driver: 51 - J.J. Yeley / In-Active>, <Driver: 84 - Jimmie Johnson / Legacy Motor Club>, <Driver: 22 - Joey Logano / Team Penske>, <Driver: 42 - John Hunter Nemechek / Legacy Motor Club>, '...(remaining elements truncat

### 5. How to do union of two querysets from same or different models?
5. How to do union of two querysets from same or different models?

https://books.agiliq.com/projects/django-orm-cookbook/en/latest/notequal_query.html

In [19]:
bob = Bet.objects.filter(race__race_date='2024-03-10').values_list('player__player_name','race__race_date','driver__name','finish')
greg = Bet.objects.filter(Q(race__race_date='2024-03-10') & Q(player__player_name = 'Greg')).values_list('player__player_name','race__race_date','driver__name','finish')
bob.union(greg)

race1 = Race.objects.all().values_list('race_date')
bets = Bet.objects.all().values('race__race_date')
print(race1)
print(bets)
race1.union(bets)

<QuerySet [(datetime.date(2022, 3, 6),), (datetime.date(2023, 3, 5),), (datetime.date(2023, 3, 19),), (datetime.date(2023, 3, 26),), (datetime.date(2024, 2, 18),), (datetime.date(2024, 2, 25),), (datetime.date(2024, 3, 3),), (datetime.date(2024, 3, 10),), (datetime.date(2024, 3, 17),), (datetime.date(2024, 3, 24),), (datetime.date(2024, 3, 31),), (datetime.date(2024, 4, 7),), (datetime.date(2024, 4, 14),)]>
<QuerySet [{'race__race_date': datetime.date(2024, 2, 18)}, {'race__race_date': datetime.date(2024, 2, 18)}, {'race__race_date': datetime.date(2024, 2, 25)}, {'race__race_date': datetime.date(2024, 2, 25)}, {'race__race_date': datetime.date(2024, 3, 3)}, {'race__race_date': datetime.date(2024, 3, 3)}]>


<QuerySet [(datetime.date(2022, 3, 6),), (datetime.date(2023, 3, 5),), (datetime.date(2023, 3, 19),), (datetime.date(2023, 3, 26),), (datetime.date(2024, 2, 18),), (datetime.date(2024, 2, 25),), (datetime.date(2024, 3, 3),), (datetime.date(2024, 3, 10),), (datetime.date(2024, 3, 17),), (datetime.date(2024, 3, 24),), (datetime.date(2024, 3, 31),), (datetime.date(2024, 4, 7),), (datetime.date(2024, 4, 14),)]>